  # Agente de Estudos

*Este notebook irá guiá-lo pelos fundamentos de construção de aplicações usando LLMs e a biblioteca LangChain em Python. O objetivo é criar um agente de estudo para auxiliar estudantes que estão se preparando para o Exame Nacional do Ensino Médio (ENEM). A ideia principal é que o agente pergunte aos estudantes sobre o nível de dificuldade em tópicos de Geofísica, calcule quanto tempo de estudo é necessário para cada tópico e, ao final, permita que os estudantes informem seu progresso e percentual de evolução.*

Antes de começar, precisamos instalar todas as dependências necessárias para este agente.

In [1]:
!pip install openai langchain pandas ipywidgets
!pip install --upgrade langchain openai langchain[llms] --quiet
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
goo

Importe as bibliotecas e forneça sua chave da OpenAI de forma segura. Não substitua nada manualmente, apenas execute a célula e digite sua chave quando solicitado.

In [3]:
import os
from getpass import getpass
import pandas as pd
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from IPython.display import HTML, display

import ipywidgets as widgets
from IPython.display import display

os.environ["OPENAI_API_KEY"] = getpass("Digite Sua Key: ")

llm = OpenAI(temperature=0.7)


Digite Sua Key: ··········


/tmp/ipython-input-1144235858.py:14: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)


Inicializa os assuntos e tópicos de estudo, definindo dificuldade e progresso iniciais.

In [4]:
subjects = [
    {"subject": "Geopolitica", "topics": [
        {"name": "Globalizacao", "difficulty":0, "progress":0.0},
        {"name": "Blocos economicos", "difficulty":0, "progress":0.0},
        {"name": "Geopolitica brasileira", "difficulty":0, "progress":0.0}
    ]}
]

Aqui criamos sliders interativos para o estudante informar a dificuldade de cada tópico e salvar todas as respostas.

In [5]:
def ask_difficulty_colab(subjects):
    display(HTML("<h2>Dificuldade de cada tópico (1=fácil, 5=muito difícil)</h2>"))

    sliders, topics_list = [], []
    for subj in subjects:
        display(HTML(f"<h3>{subj['subject']}</h3>"))
        for topic in subj["topics"]:
            slider = widgets.IntSlider(value=3, min=1, max=5, description=topic['name'])
            display(slider)
            sliders.append(slider)
            topics_list.append(topic)

    btn = widgets.Button(description="Confirmar")
    display(btn)

    def confirm(b):
        for s, t in zip(sliders, topics_list):
            t['difficulty'] = s.value
            s.disabled = True
        b.disabled = True
        display(HTML("<b>Dificuldades atualizadas!</b>"))

    btn.on_click(confirm)
    return subjects


Já aqui o estudante deve informar o progresso de cada tópico e salva todas as respostas.

In [6]:
def update_progress_colab(subjects):
    display(HTML("<h2>Progresso de cada tópico (0 a 100%)</h2>"))

    sliders, topics_list = [], []
    for subj in subjects:
        display(HTML(f"<h3>{subj['subject']}</h3>"))
        for topic in subj['topics']:
            slider = widgets.FloatSlider(value=topic['progress']*100, min=0, max=100, step=1, description=topic['name'])
            display(slider)
            sliders.append(slider)
            topics_list.append(topic)

    btn = widgets.Button(description="Confirmar")
    display(btn)

    def confirm(b):
        for s, t in zip(sliders, topics_list):
            t['progress'] = s.value / 100
            s.disabled = True
        b.disabled = True
        display(HTML("<b>Progresso atualizado!</b>"))

    btn.on_click(confirm)
    return subjects



Aqui temos uma função de converter valor decimal dehoras e minutos (por exemplo, 2.5 → 2h 30m).

In [7]:
def decimal_to_hours_minutes(hours_decimal):
    horas = int(hours_decimal)
    minutos = int(round((hours_decimal - horas) * 60))
    return f"{horas}h {minutos}m"

Essa função serve para calcular quanto tempo estudar cada tópico com base em dificuldade e progresso, e mostra o resultado em uma tabela

In [8]:
def generate_daily_schedule(subjects, available_hours=4):
    topic_list = []
    for subject in subjects:
        for topic in subject["topics"]:
            priority = topic["difficulty"] * (1 - topic.get("progress",0))
            topic_list.append({**topic, "subject": subject["subject"], "priority": priority})

    total_priority = sum(t["priority"] for t in topic_list)
    if total_priority == 0:
        display(HTML("<b>Todos os tópicos já concluídos!</b>"))
        return []

    schedule = []
    for t in topic_list:
        hours_decimal = available_hours * (t["priority"] / total_priority)
        schedule.append({
            "subject": t["subject"],
            "topic": t["name"],
            "daily_hours": decimal_to_hours_minutes(hours_decimal)
        })

    df_schedule = pd.DataFrame(schedule)
    display(df_schedule)
    return schedule

Os assuntos e tópicos são covertidos em um texto formatado, mostrando dificuldade e progresso de cada tópico

In [31]:
def subjects_to_text(subjects):
    text = ""
    for subj in subjects:
        text += f"{subj['subject']}:\n"
        for topic in subj['topics']:
            text += f"- {topic['name']} (Dificuldade: {topic['difficulty']}, Progresso: {topic['progress']*100:.0f}%)\n"
    return text

Define as instruções para o agente de estudo: pede para ele olhar os tópicos do estudante, sugerir quais estudar primeiro, quanto tempo dedicar a cada um e dar dicas rápidas de estudo.


In [10]:
prompt_template = """
Você é um agente de estudo para o ENEM. O estudante tem os seguintes tópicos:

{subjects_text}

Baseado nisso, sugira:
1. Quais tópicos priorizar hoje.
2. Quanto tempo estudar em cada tópico.
3. Dicas rápidas de estudo.
"""

prompt = PromptTemplate(input_variables=["subjects_text"], template=prompt_template)

Gera a estratégia de estudo personalizada usando o LLM, com base nos tópicos e progresso do estudante.

In [24]:
def generate_study_strategy(subjects):
    subjects_text = subjects_to_text(subjects)
    chain = LLMChain(llm=llm, prompt=prompt)
    try:
        response = chain.run(subjects_text=subjects_text)
        display(HTML(f"<h3>Estratégia do LLM:</h3><p>{response}</p>"))
    except Exception as e:
        print("Erro:", e)


E por fim a chamada das funções

In [33]:
# 1. Perguntar dificuldade
subjects = ask_difficulty_colab(subjects)

# 2. Perguntar progresso
subjects = update_progress_colab(subjects)

# 3. Gerar agenda diária (6 horas disponíveis, por exemplo)
schedule = generate_daily_schedule(subjects, available_hours=6)

# # 4. Gerar estratégia personalizada via LangChain
generate_study_strategy(subjects)


IntSlider(value=3, description='Globalizacao', max=5, min=1)

IntSlider(value=3, description='Blocos economicos', max=5, min=1)

IntSlider(value=3, description='Geopolitica brasileira', max=5, min=1)

Button(description='Confirmar', style=ButtonStyle())

FloatSlider(value=0.0, description='Globalizacao', step=1.0)

FloatSlider(value=100.0, description='Blocos economicos', step=1.0)

FloatSlider(value=67.0, description='Geopolitica brasileira', step=1.0)

Button(description='Confirmar', style=ButtonStyle())

,subject,topic,daily_hours
0,Geopolitica,Globalizacao,4h 31m
1,Geopolitica,Blocos economicos,0h 0m
2,Geopolitica,Geopolitica brasileira,1h 29m


Erro: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
